<a href="https://colab.research.google.com/github/Tzoll87/Stock/blob/master/Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.8 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 5.2 MB/s 
     |████████████████████████████████| 357 kB 40.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as wb
import yfinance as yf
import dash
import plotly.express as px
import datetime
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
end = date.today() - datetime.timedelta(1)

In [ ]:
#Creating a dictionary based on stock in portfolio, the amount of them, and the average buying price
dik = {'Stock':['FB', 'TTD', 'DIS', 'WYNN', 'NVDA', 'DAL', 'SDC', 'U', 'UAL', 'PARA', 'RCL', 'PLTR', 'BABA', 'CCL', 'CRSR', 'TTCF', 'TDOC'], 'Value': [171.3587, 52.0804, 114.4821, 77.85343, 218.2, 54.21, 2.41283, 89.0887, 74.78,41.22377, 118.6043, 16.77974, 167.5725, 29.81967, 29.5272, 17.08706, 79.8], 'Amount': [7.00286, 29.76045, 7.68679, 28.90046, 2.79204, 7.19425, 257.3739, 23.23527, 3.74432, 53.65836, 6.07061, 274.7361, 6.92238, 70.42332, 92.45712, 259.8458, 7.5188]} 

In [ ]:
data_dict = pd.DataFrame.from_dict(dik)

In [ ]:
data_dict.head()

,Stock,Value,Amount
0,FB,171.35870,7.00286
1,TTD,52.08040,29.76045
2,DIS,114.48210,7.68679
3,WYNN,77.85343,28.90046
4,NVDA,218.20000,2.79204


In [ ]:
tickers = ['FB', 'TTD', 'DIS', 'WYNN', 'NVDA', 'DAL', 'SDC', 'U', 'UAL', 'PARA', 'RCL', 'PLTR', 'BABA', 'CCL', 'CRSR', 'TTCF', 'TDOC']

In [ ]:
#From yahoo finance getting the close price of the selected stocks until yesterday
new_data = pd.DataFrame()
for i in tickers:
  new_data[i] = yf.download(i, start = '2022-04-25', end=end)['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
new_data = new_data.reset_index()

In [ ]:
new_data = pd.melt(new_data, id_vars= 'Date', var_name='Stock', value_name='Value')

In [ ]:
new_data.head()

,Date,Stock,Value
0,2022-04-25,FB,186.990005
1,2022-04-26,FB,180.949997
2,2022-04-27,FB,174.949997
3,2022-04-28,FB,205.729996
4,2022-04-29,FB,200.470001


In [ ]:
#Merging dataset to create one which contains the day-by-day changes of the stock prices & the starting portfolio distribution info
df = new_data.merge(data_dict, left_on = 'Stock', right_on = 'Stock', how = 'inner')

In [ ]:
df['Diff'] = df['Value_x'] - df['Value_y']

In [ ]:
df['Actual'] = df['Diff'] * df['Amount']

In [ ]:
df['Current'] = df['Value_x'] * df['Amount']

In [ ]:
#Creating a view for a daily portfolio value
df_grouped = df.groupby('Date').sum()['Current'].reset_index()

In [ ]:
df_grouped.head()

,Date,Current
0,2022-04-25,21272.742795
1,2022-04-26,20050.626870
2,2022-04-27,19624.808914
3,2022-04-28,20125.926412
4,2022-04-29,19598.864291


In [ ]:
#Variable to evaluate the starter value of portfolio and generating later a view in which I look at how big is the portfoli compared to:
#a) starting value of the portfolio
#b) to create a view where I can compare portfolio size movement with the S&P500 & NASDAQ indexes
szum = (data_dict['Value'] * data_dict['Amount']).sum()

In [ ]:
df_grouped['Pct_value'] = df_grouped['Current'] / szum * 100

In [ ]:
sp_nasd = pd.DataFrame()
for i in ['^GSPC', '^IXIC']:
  sp_nasd[i] = yf.download(i, start = '2022-04-25', end=end)['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
sp_nasd = sp_nasd.reset_index()

In [ ]:
sp_nasd['SP_change'] = sp_nasd['^GSPC'] / sp_nasd.iloc[0, 1] *100
sp_nasd['ND_change'] = sp_nasd['^IXIC'] / sp_nasd.iloc[0, 2] *100

In [ ]:
df_final = pd.merge(df_grouped, sp_nasd, on = 'Date')

In [ ]:
df_final = df_final.loc[:, ['Date', 'Pct_value', 'SP_change', 'ND_change']]

In [ ]:
fig = px.line(df_final, x="Date", y=df_final.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title='Portfolio performance comparison')
fig.update_xaxes(
    dtick="M1",
    tickformat="%d\n%b\n%Y",
    ticklabelmode="period")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
df_stock = df.pivot_table(index = 'Date', columns = 'Stock', values = 'Current')

In [ ]:
#Area charts by stock to track the change in stock value withing portfolio
fig = px.area(df_stock, facet_col="Stock", facet_col_wrap=3)
fig.show()

In [ ]:
graph = df_stock.reset_index()

In [ ]:
fig = px.line(graph, x="Date", y=tickers,
              hover_data={"Date": "|%B %d, %Y"},
              title='custom tick labels')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

In [ ]:
starter = pd.DataFrame(graph.iloc[1,1:]).reset_index()
ender = pd.DataFrame(graph.iloc[-1,1:]).reset_index()
ender.columns = ['Stock', 'Value']

In [ ]:
#Pie charts to show the distribution of stock at the start of the portfolio and on the end of the examined time period
fig = px.pie(starter, values=1, names='Stock', title='Starting stock portfolio size')
fig2 = px.pie(ender, values='Value', names='Stock', title='Current stock portfolio size')
fig.show()
fig2.show()

In [ ]:
#KPI cards to symbolize porfolio change by value and by percentage for certain time periods (all-time, compared to 1 week and 1 month ago)
fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = ender['Value'].sum(),
    number = {'prefix': "$"},
    delta = {'position': "top", 'reference': szum},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig.update_layout(paper_bgcolor = "lightgray")

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = ender['Value'].sum(),
    title = {"text": "1 week change<br><span style='font-size:0.8em;color:gray'>"},
    domain = {'x': [0, 0.5], 'y': [0, 0.5]},
    delta = {'reference': Week, 'relative': True, 'position' : "bottom"}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = ender['Value'].sum(),
    title = {"text": "1 month change<br><span style='font-size:0.8em;color:gray'>"},
    delta = {'reference': Month, 'relative': True},
    domain = {'x': [0.6, 1], 'y': [0, 0.5]}))

fig.show()

In [ ]:
cards = df.groupby('Date').sum()['Current'].reset_index()

In [ ]:
cards.head()

,Date,Current
0,2022-04-25,21272.742795
1,2022-04-26,20050.626870
2,2022-04-27,19624.808914
3,2022-04-28,20125.926412
4,2022-04-29,19598.864291


In [ ]:
Week = cards.iloc[-7,1]
Month = cards.iloc[-30,1]


In [ ]:
#Creating a more complex dataset for candlestick visualization stock-by-stock
stock_df = yf.download(tickers, start = '2022-04-25', end=end, group_by = 'ticker')

[*********************100%***********************]  17 of 17 completed


In [ ]:
stock_df.head()

,Date,level_1,Adj Close,Close,High,Low,Open,Volume
0,2022-04-25,BABA,85.839996,85.839996,86.169998,81.800003,82.699997,24674000
1,2022-04-25,CCL,18.879999,18.879999,18.889999,18.340000,18.730000,27164700
2,2022-04-25,CRSR,16.090000,16.090000,16.209999,15.690000,15.900000,1205900
3,2022-04-25,DAL,43.270000,43.270000,43.549999,42.230000,43.400002,15156900
4,2022-04-25,DIS,119.949997,119.949997,120.330002,116.750000,117.660004,16491000


In [ ]:
stock_df = stock_df.stack(0).reset_index()

In [ ]:
for item in tickers:
  filtered = stock_df[stock_df['level_1'] == item]
  fig = go.Figure(data=[go.Candlestick(x=filtered['Date'], open=filtered['Open'],
                high=filtered['High'],
                low=filtered['Low'],
                close=filtered['Close'])])
  fig.update_layout(
    title= item + ' Candlestick chart',
    yaxis_title= item + ' Stock Price')

  fig.show()